In [78]:
import torch
import torchvision
from torchvision.models import resnet50
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

In [79]:
CLASSES = 2  # Two classes: grass and flower
BATCH_SIZE = 32
NUM_EPOCHS = 10
LEARNING_RATE = 0.001
CHECKPOINT_PATH = 'resnet50_weights_best_acc.tar'
DATASET_ROOT = 'dataset/'

In [80]:
# Set up TensorBoard
log_dir = f'runs/grass_flower_{datetime.now().strftime("%Y%m%d-%H%M%S")}'
writer = SummaryWriter(log_dir)

In [81]:
# Load the model
model = resnet50(weights=None, )  # Load without pre-trained weights
model.fc = torch.nn.Linear(model.fc.in_features, 1081)  # Adjust for binary classification

# Set Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Check CUDA 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(device)

cpu


In [82]:
# Load the checkpoint
checkpoint = torch.load(CHECKPOINT_PATH, map_location=device, weights_only=True)
print("Checkpoint keys:", checkpoint.keys())

model.load_state_dict(checkpoint['model'])  # Load model weights

Checkpoint keys: dict_keys(['epoch', 'model', 'optimizer'])


<All keys matched successfully>

In [83]:
model.fc = torch.nn.Linear(model.fc.in_features, CLASSES)  # Adjust for binary classification

In [84]:
# Prepare the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match ResNet input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ResNet normalization
])

In [85]:
train_dataset = datasets.ImageFolder(root=f'{DATASET_ROOT}train', transform=transform)
val_dataset = datasets.ImageFolder(root=f'{DATASET_ROOT}val', transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [86]:
# Training
criterion = torch.nn.CrossEntropyLoss()

# Training loop, continuing from start_epoch
for epoch in range(NUM_EPOCHS):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()  # Clear previous gradients
        loss.backward()        # Backpropagation
        optimizer.step()       # Update weights
        
        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
    writer.add_scalar('Loss/train', avg_loss, epoch)

    # Validation phase (optional)
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_loss /= len(val_loader)
    val_accuracy = 100 * correct / total
    writer.add_scalar('Loss/val', val_loss, epoch)
    writer.add_scalar('Accuracy/val', val_accuracy, epoch)

    print(f"Epoch [{epoch + 1}/{NUM_EPOCHS}], Train Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

# Save the model after training
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'epoch': NUM_EPOCHS,
}, 'resnet50_retrained_grass_flower.pth')

# Close TensorBoard writer
writer.close()

KeyboardInterrupt: 